# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

CustomerID FirstName   LastName  SalesID  ProductID           ProductName  \
0       61288      Rosa   Andersen   134196        229  Bread - Hot Dog Buns   
1       77352     Myron     Murray  6167892        229  Bread - Hot Dog Buns   
2       40094     Susan  Stevenson  5970885        229  Bread - Hot Dog Buns   
3       23548    Tricia    Vincent  6426954        229  Bread - Hot Dog Buns   
4       78981     Scott      Burch   819094        229  Bread - Hot Dog Buns   

   Quantity  
0        16  
1        20  
2        11  
3         6  
4        20

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [30]:
grouped = data.groupby(['CustomerID', 'ProductName']).sum()
grouped

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1
...                                           ...        ...       ...
98200      Vol Au Vents                   5160346        272        50
           Wasabi Powder                  4438427         80        25
           Wine - Fume Blanc Fetzer       6226060         76        25
           Wine - Hardys Bankside Shiraz   266676        360        25
           Yogurt - French Vanilla        2112061        320        25

[63628 rows x 3 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [21]:
matrix = grouped.pivot_table('Quantity', 'ProductName', 'CustomerID', aggfunc='sum', fill_value = 0)
matrix

CustomerID                         33     200    264    356    412    464    \
ProductName                                                                   
Anchovy Paste - 56 G Tube              0      0      0      0      0      0   
Appetizer - Mini Egg Roll, Shrimp      0      0      0      0      0      0   
Appetizer - Mushroom Tart              0      0      0      0      0      0   
Appetizer - Sausage Rolls              0      0      0      0      0      0   
Apricots - Dried                       1      0      0      0      1      0   
...                                  ...    ...    ...    ...    ...    ...   
Yeast Dry - Fermipan                   0      0      0      0      0      0   
Yoghurt Tubes                          0      0      0      0      0      0   
Yogurt - Blueberry, 175 Gr             0      1      0      0      0      0   
Yogurt - French Vanilla                1      0      0      1      0      0   
Zucchini - Yellow                      0      0      0      0      0      0   

CustomerID                         477    639    649    669    ...  97697  \
ProductName                                                    ...          
Anchovy Paste - 56 G Tube              0      1      0      0  ...      0   
Appetizer - Mini Egg Roll, Shrimp      0      0      0      0  ...     25   
Appetizer - Mushroom Tart              0      1      0      0  ...     25   
Appetizer - Sausage Rolls              0      0      0      0  ...      0   
Apricots - Dried                       0      0      0      0  ...      0   
...                                  ...    ...    ...    ...  ...    ...   
Yeast Dry - Fermipan                   0      0      0      0  ...      0   
Yoghurt Tubes                          0      0      2      0  ...      0   
Yogurt - Blueberry, 175 Gr             0      0      0      0  ...     25   
Yogurt - French Vanilla                2      0      0      1  ...      0   
Zucchini - Yellow                      0      0      0      0  ...      0   

CustomerID                         97753  97769  97793  97900  97928  98069  \
ProductName                                                                   
Anchovy Paste - 56 G Tube             25      0      0      0      0      0   
Appetizer - Mini Egg Roll, Shrimp     25      0      0      0      0      0   
Appetizer - Mushroom Tart              0      0      0      0      0      0   
Appetizer - Sausage Rolls              0      0      0     25     25     25   
Apricots - Dried                      25      0      0      0      0      0   
...                                  ...    ...    ...    ...    ...    ...   
Yeast Dry - Fermipan                   0      0     25      0      0      0   
Yoghurt Tubes                          0      0      0      0      0      0   
Yogurt - Blueberry, 175 Gr             0      0     25      0      0      0   
Yogurt - French Vanilla                0     25      0      0      0      0   
Zucchini - Yellow                      0      0      0      0      0      0   

CustomerID                         98159  98185  98200  
ProductName                                             
Anchovy Paste - 56 G Tube              0      0      0  
Appetizer - Mini Egg Roll, Shrimp      0      0      0  
Appetizer - Mushroom Tart              0     25      0  
Appetizer - Sausage Rolls              0     25      0  
Apricots - Dried                       0      0      0  
...                                  ...    ...    ...  
Yeast Dry - Fermipan                   0      0      0  
Yoghurt Tubes                         25      0      0  
Yogurt - Blueberry, 175 Gr             0      0      0  
Yogurt - French Vanilla                0      0     25  
Zucchini - Yellow                      0      0      0  

[452 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [23]:
# I need to transpose the matrix, otherwise I get the distance for products, no customers.
# First applying pdist, gives an 1D array.
# Then applying squareform to turn it into a squareform
# Finally convert it into a DataFrame

dist_matrix = pd.DataFrame(squareform(pdist(matrix.T)), index=matrix.columns, columns=matrix.columns)
dist_matrix

CustomerID       33          200         264         356         412    \
CustomerID                                                               
33            0.000000   11.916375   10.488088   11.224972   11.401754   
200          11.916375    0.000000   11.747340   12.083046   12.569805   
264          10.488088   11.747340    0.000000   11.489125   11.224972   
356          11.224972   12.083046   11.489125    0.000000   12.083046   
412          11.401754   12.569805   11.224972   12.083046    0.000000   
...                ...         ...         ...         ...         ...   
97928       220.501701  220.202180  219.136943  219.952268  219.733930   
98069       217.188858  215.728997  216.612557  217.437347  217.446545   
98159       228.628520  228.010965  228.081126  228.098663  227.997807   
98185       239.000000  239.037654  238.266657  238.493186  238.396728   
98200       229.773802  229.704158  229.773802  229.464594  228.927936   

CustomerID       464         477         639         649         669    ...  \
CustomerID                                                              ...   
33           11.090537   12.409674   11.045361   11.269428   11.489125  ...   
200          12.288206   12.165525   12.083046   11.874342   12.000000  ...   
264          11.445523   12.000000   11.401754   11.180340   11.747340  ...   
356          11.789826   12.328828   11.135529   11.958261   12.165525  ...   
412          11.704700   12.328828   11.135529   11.789826   11.747340  ...   
...                ...         ...         ...         ...         ...  ...   
97928       219.599636  219.538152  219.924987  219.827205  220.070443  ...   
98069       217.425849  216.903204  217.294731  217.080630  216.751009  ...   
98159       228.197283  228.028507  228.181945  227.868383  228.103047  ...   
98185       239.006276  238.949786  238.468027  238.692271  239.334494  ...   
98200       229.017467  229.503813  229.656265  229.997826  229.032749  ...   

CustomerID       97697       97753       97769       97793       97900  \
CustomerID                                                               
33          206.871941  213.180675  225.656819  198.232187  230.913404   
200         206.310446  212.635839  224.697575  197.139544  230.952376   
264         206.387984  212.946003  225.435135  197.600607  230.371439   
356         206.649462  213.082144  225.452878  197.494304  231.038958   
412         206.900942  211.679002  225.572605  197.630969  230.614397   
...                ...         ...         ...         ...         ...   
97928       283.945417  283.945417  302.076149  272.717803  278.388218   
98069       283.945417  283.945417  295.803989  283.945417  285.043856   
98159       283.945417  279.508497  300.000000  290.473751  300.000000   
98185       301.039864  315.238005  306.186218  292.617498  314.245127   
98200       291.547595  297.909382  294.745653  289.395923  303.108891   

CustomerID       97928       98069       98159       98185       98200  
CustomerID                                                              
33          220.501701  217.188858  228.628520  239.000000  229.773802  
200         220.202180  215.728997  228.010965  239.037654  229.704158  
264         219.136943  216.612557  228.081126  238.266657  229.773802  
356         219.952268  217.437347  228.098663  238.493186  229.464594  
412         219.733930  217.446545  227.997807  238.396728  228.927936  
...                ...         ...         ...         ...         ...  
97928         0.000000  273.861279  291.547595  306.186218  307.205143  
98069       273.861279    0.000000  287.228132  297.909382  294.745653  
98159       291.547595  287.228132    0.000000  304.138127  305.163890  
98185       306.186218  297.909382  304.138127    0.000000  303.108891  
98200       307.205143  294.745653  305.163890  303.108891    0.000000  

[1000 rows x 1000 columns]

In [26]:
# The distances I have doesn't tell me much. I will normalize to a value between 0 and 1,
# and inverse them: The closer to 1, the more similar they are

dist_norm = pd.DataFrame(1/(1 + dist_matrix))
dist_norm

CustomerID     33        200       264       356       412       464    \
CustomerID                                                               
33          1.000000  0.077421  0.087047  0.081800  0.080634  0.082709   
200         0.077421  1.000000  0.078448  0.076435  0.073693  0.075255   
264         0.087047  0.078448  1.000000  0.080070  0.081800  0.080350   
356         0.081800  0.076435  0.080070  1.000000  0.076435  0.078187   
412         0.080634  0.073693  0.081800  0.076435  1.000000  0.078711   
...              ...       ...       ...       ...       ...       ...   
97928       0.004515  0.004521  0.004543  0.004526  0.004530  0.004533   
98069       0.004583  0.004614  0.004595  0.004578  0.004578  0.004578   
98159       0.004355  0.004367  0.004365  0.004365  0.004367  0.004363   
98185       0.004167  0.004166  0.004179  0.004175  0.004177  0.004167   
98200       0.004333  0.004335  0.004333  0.004339  0.004349  0.004347   

CustomerID     477       639       649       669    ...     97697     97753  \
CustomerID                                          ...                       
33          0.074573  0.083020  0.081503  0.080070  ...  0.004811  0.004669   
200         0.075956  0.076435  0.077674  0.076923  ...  0.004824  0.004681   
264         0.076923  0.080634  0.082100  0.078448  ...  0.004822  0.004674   
356         0.075025  0.082403  0.077171  0.075956  ...  0.004816  0.004671   
412         0.075025  0.082403  0.078187  0.078448  ...  0.004810  0.004702   
...              ...       ...       ...       ...  ...       ...       ...   
97928       0.004534  0.004526  0.004528  0.004523  ...  0.003509  0.003509   
98069       0.004589  0.004581  0.004585  0.004592  ...  0.003509  0.003509   
98159       0.004366  0.004363  0.004369  0.004365  ...  0.003509  0.003565   
98185       0.004168  0.004176  0.004172  0.004161  ...  0.003311  0.003162   
98200       0.004338  0.004335  0.004329  0.004347  ...  0.003418  0.003345   

CustomerID     97769     97793     97900     97928     98069     98159  \
CustomerID                                                               
33          0.004412  0.005019  0.004312  0.004515  0.004583  0.004355   
200         0.004431  0.005047  0.004311  0.004521  0.004614  0.004367   
264         0.004416  0.005035  0.004322  0.004543  0.004595  0.004365   
356         0.004416  0.005038  0.004310  0.004526  0.004578  0.004365   
412         0.004414  0.005034  0.004318  0.004530  0.004578  0.004367   
...              ...       ...       ...       ...       ...       ...   
97928       0.003300  0.003653  0.003579  1.000000  0.003638  0.003418   
98069       0.003369  0.003509  0.003496  0.003638  1.000000  0.003469   
98159       0.003322  0.003431  0.003322  0.003418  0.003469  1.000000   
98185       0.003255  0.003406  0.003172  0.003255  0.003345  0.003277   
98200       0.003381  0.003444  0.003288  0.003245  0.003381  0.003266   

CustomerID     98185     98200  
CustomerID                      
33          0.004167  0.004333  
200         0.004166  0.004335  
264         0.004179  0.004333  
356         0.004175  0.004339  
412         0.004177  0.004349  
...              ...       ...  
97928       0.003255  0.003245  
98069       0.003345  0.003381  
98159       0.003277  0.003266  
98185       1.000000  0.003288  
98200       0.003288  1.000000  

[1000 rows x 1000 columns]

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [29]:
Top5_cust200 = dist_norm[200].sort_values(ascending = False).head(6)
Top5_cust200

CustomerID
200     1.000000
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [54]:
# I select index from 1 because I don't want to get the first input, 
# as it is the customer itself

similar = grouped.loc[(Top5_cust200.index[1:],)]
similar

SalesID  ProductID  Quantity
CustomerID ProductName                                                    
1072       Appetizer - Mini Egg Roll, Shrimp  4770059        327         1
           Appetizer - Sausage Rolls          6604526         73         1
           Beans - Kidney, Canned             1148109        300         1
           Beef - Montreal Smoked Brisket      705042        373         1
           Beef - Striploin Aa                1804237        339         1
...                                               ...        ...       ...
3909       Wine - Charddonnay Errazuriz       9045484        450         2
           Wine - Fume Blanc Fetzer           3349679         76         1
           Wine - Magnotta, Merlot Sr Vqa     3566710        192         1
           Wine - Toasted Head                4134000        134         1
           Wine - Valpolicella Masi           3584400        296         1

[262 rows x 3 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [57]:
agg_similar = similar.groupby('ProductName')[['Quantity']].sum()\
                .sort_values(by = 'Quantity', ascending = False)
agg_similar

Quantity
ProductName                              
Soup - Campbells Bean Medley            4
Muffin - Carrot Individual Wrap         3
Bay Leaf                                3
Pork - Kidney                           3
Pepper - Black, Whole                   3
...                                   ...
Kiwi                                    1
Knife Plastic - White                   1
Lambcasing                              1
Lentils - Red, Dry                      1
Yoghurt Tubes                           1

[207 rows x 1 columns]

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [82]:
products = pd.concat([agg_similar, matrix[200]], axis=1, sort=False)
products.rename(columns = {200:'Cust_200'}, inplace = True)
products

Quantity  Cust_200
Soup - Campbells Bean Medley           4.0         0
Muffin - Carrot Individual Wrap        3.0         0
Bay Leaf                               3.0         0
Pork - Kidney                          3.0         0
Pepper - Black, Whole                  3.0         1
...                                    ...       ...
Wine - White, Mosel Gold               NaN         0
Wine - White, Schroder And Schyl       NaN         0
Yogurt - Blueberry, 175 Gr             NaN         1
Yogurt - French Vanilla                NaN         0
Zucchini - Yellow                      NaN         0

[452 rows x 2 columns]

In [86]:
Top5rec = products.query('Quantity > 0 and Cust_200 == 0').head(5)
Top5rec

Quantity  Cust_200
Soup - Campbells Bean Medley          4.0         0
Muffin - Carrot Individual Wrap       3.0         0
Bay Leaf                              3.0         0
Pork - Kidney                         3.0         0
Wanton Wrap                           3.0         0

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [88]:
recommendations = {}
unique_ID = dist_norm.columns.unique()

In [99]:
for customer in unique_ID:
    head = dist_norm[customer].sort_values(ascending = False).head(6)
    similar = grouped.loc[(head.index[1:],)]
    agg_similar = similar.groupby('ProductName')[['Quantity']].sum()\
                .sort_values(by = 'Quantity', ascending = False)
    products = pd.concat([agg_similar, matrix[customer]], axis=1, sort=False)
    products.rename(columns = {customer:'customer'}, inplace = True)
    recommendations[customer] = list(products.query('Quantity > 0 and customer == 0').head(5).index)
    

In [100]:
recommendations

{33: ['Butter - Unsalted',
  'Wine - Ej Gallo Sierra Valley',
  'Soup - Campbells Bean Medley',
  'Wine - Blue Nun Qualitatswein',
  'Chicken - Soup Base'],
 200: ['Soup - Campbells Bean Medley',
  'Muffin - Carrot Individual Wrap',
  'Bay Leaf',
  'Pork - Kidney',
  'Wanton Wrap'],
 264: ['Soupfoamcont12oz 112con',
  'Wine - Two Oceans Cabernet',
  'Bread - Italian Roll With Herbs',
  'Veal - Inside, Choice',
  'Fish - Scallops, Cold Smoked'],
 356: ['Veal - Inside, Choice',
  'Lamb - Ground',
  'Pomello',
  'Butter - Unsalted',
  'Tart Shells - Sweet, 4'],
 412: ['Olive - Spread Tapenade',
  'Sprouts - Baby Pea Tendrils',
  'Wine - Blue Nun Qualitatswein',
  'Pepper - Black, Whole',
  'Soup - Campbells Bean Medley'],
 464: ['Butter - Unsalted',
  'Sauce - Gravy, Au Jus, Mix',
  'Pepper - Black, Whole',
  'Peas - Pigeon, Dry',
  'Bar - Granola Trail Mix Fruit Nut'],
 477: ['Olive - Spread Tapenade',
  'Sprouts - Baby Pea Tendrils',
  'Wine - Blue Nun Qualitatswein',
  'Veal - Inside, 

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [103]:
recommendations_df = pd.DataFrame.from_dict(recommendations, orient='index', 
                                columns=['rec1', 'rec2', 'rec3', 'rec4', 'rec5'])
recommendations_df

rec1                             rec2  \
33                Butter - Unsalted    Wine - Ej Gallo Sierra Valley   
200    Soup - Campbells Bean Medley  Muffin - Carrot Individual Wrap   
264         Soupfoamcont12oz 112con       Wine - Two Oceans Cabernet   
356           Veal - Inside, Choice                    Lamb - Ground   
412         Olive - Spread Tapenade      Sprouts - Baby Pea Tendrils   
...                             ...                              ...   
97928      Soup - Campbells, Lentil             Cheese - Brie,danish   
98069               Skirt - 29 Foot             Beans - Kidney White   
98159           Lamb - Whole, Fresh       Bread - Raisin Walnut Oval   
98185               Crackers - Trio                           Pernod   
98200                   Beans - Wax                     Sherry - Dry   

                                  rec3                           rec4  \
33        Soup - Campbells Bean Medley  Wine - Blue Nun Qualitatswein   
200                           Bay Leaf                  Pork - Kidney   
264    Bread - Italian Roll With Herbs          Veal - Inside, Choice   
356                            Pomello              Butter - Unsalted   
412      Wine - Blue Nun Qualitatswein          Pepper - Black, Whole   
...                                ...                            ...   
97928                Wiberg Super Cure         Pants Custom Dry Clean   
98069                        Milk - 1%       Cheese - Taleggio D.o.p.   
98159    Chips Potato Salt Vinegar 43g        Tea - English Breakfast   
98185               Tea - Jasmin Green        Pastry - Choclate Baked   
98200    Pork - Hock And Feet Attached                   Pears - Bosc   

                               rec5  
33              Chicken - Soup Base  
200                     Wanton Wrap  
264    Fish - Scallops, Cold Smoked  
356          Tart Shells - Sweet, 4  
412    Soup - Campbells Bean Medley  
...                             ...  
97928                Shrimp - 31/40  
98069   Sprouts - Baby Pea Tendrils  
98159       Wine - Redchard Merritt  
98185            Peas - Pigeon, Dry  
98200    Soup - Campbells, Cream Of  

[1000 rows x 5 columns]

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.